In [5]:
import numpy as np
from lab3_proto import *
from lab3_tools import *
import sys  
sys.path.append('/home/tmh/Documents/yhu/speech/Speech-Speaker-Recognition/lab1')
from lab1_proto import *
sys.path.append('/home/tmh/Documents/yhu/speech/Speech-Speaker-Recognition/lab2')
from lab2_tools import *
from lab2_proto import *
from prondict import *

In [4]:
path2info('tidigits/disc_4.1.1/tidigits/train/man/ae/1a.wav')

('man', 'ae', '1', 'a')

In [5]:
loadAudio('tidigits/disc_4.1.1/tidigits/train/man/ae/z9z6531a.wav')

(array([11, 13, 11, ...,  9,  9,  9], dtype=int16), 20000)

## Target Class Definition

In [6]:
statelist = []
phoneHMMs = np.load('/home/tmh/Documents/yhu/speech/Speech-Speaker-Recognition/lab2/lab2_models_all.npz',allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone:phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' +str(id) for ph in phones for id in range(nstates[ph])]
statelist = stateList 

In [8]:
stateList.index('ay_2')

8

## Forced Alignment

In [2]:
filename ='tidigits/disc_4.1.1/tidigits/train/man/nw/z43a.wav'
samples, samplingrate = loadAudio(filename)
lmcc = mfcc(samples)

In [3]:
wordTrans = list(path2info(filename)[2])
wordTrans

['z', '4', '3']

In [4]:
phoneTrans = words2phones(wordTrans,prondict)
phoneTrans

['sil',
 'z',
 'iy',
 'r',
 'ow',
 'sp',
 'f',
 'ao',
 'r',
 'sp',
 'th',
 'r',
 'iy',
 'sp',
 'sil']

In [8]:
utteranceHMM = concatHMMs(phoneHMMs,phoneTrans)
utteranceHMM

{'name': 'silziyrowspfaorspthriyspsil',
 'startprob': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 'transmat': array([[0.8833106 , 0.1166894 , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.9164465 , 0.08355348, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.802877  , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.9164465 , 0.08355348,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.802877  ,
         0.197123  ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ]]),
 'means': array([[ 1.929856e+02, -1.658209e+01, -2.290160e+01,  3.132330e+01,
          4.658450e+01, -1.151792e+00,  1.339505e+01,  6.989146e+01,
          6